# Predicting house prices using k-nearest neighbors regression
In this notebook, you will implement k-nearest neighbors regression. You will:
  * Find the k-nearest neighbors of a given query input
  * Predict the output for the query input using the k-nearest neighbors
  * Choose the best value of k using a validation set

# Fire up GraphLab Create

In [1]:
import graphlab

# Load in house sales data

For this notebook, we use a subset of the King County housing dataset created by randomly selecting 40% of the houses in the full dataset.

In [2]:
sales = graphlab.SFrame('kc_house_data_small.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to pari.rajaram@gmail.com and will expire on January 03, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1489255138.log


# Import useful functions from previous notebooks

To efficiently compute pairwise distances among data points, we will convert the SFrame into a 2D Numpy array. First import the numpy library and then copy and paste `get_numpy_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe.select_columns(features)
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

We will also need the `normalize_features()` function from Week 5 that normalizes all feature columns to unit norm. Paste this function below.

In [5]:
def normalize_features(feature_matrix):
    norm = np.linalg.norm(feature_matrix, axis=0)
    features = feature_matrix/norm
    return features, norm

# Split data into training, test, and validation sets

In [6]:
(train_and_validation, test) = sales.random_split(.8, seed=1) # initial train/test split
(train, validation) = train_and_validation.random_split(.8, seed=1) # split training set into training and validation sets

# Extract features and normalize

Using all of the numerical inputs listed in `feature_list`, transform the training, test, and validation SFrames into Numpy arrays:

In [7]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train, feature_list, 'price')
features_test, output_test = get_numpy_data(test, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation, feature_list, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [8]:
features_train, norms = normalize_features(features_train) # normalize training set features (columns)
features_test = features_test / norms # normalize test set by training set norms
features_valid = features_valid / norms # normalize validation set by training set norms

# Compute a single distance

To start, let's just explore computing the "distance" between two given houses.  We will take our **query house** to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1.

In [9]:
features_test[0]

array([ 0.01345102,  0.01551285,  0.01807473,  0.01759212,  0.00160518,
        0.017059  ,  0.        ,  0.05102365,  0.0116321 ,  0.01564352,
        0.01362084,  0.02481682,  0.01350306,  0.        ,  0.01345386,
       -0.01346927,  0.01375926,  0.0016225 ])

In [10]:
features_train[9]

array([ 0.01345102,  0.01163464,  0.00602491,  0.0083488 ,  0.00050756,
        0.01279425,  0.        ,  0.        ,  0.01938684,  0.01390535,
        0.0096309 ,  0.        ,  0.01302544,  0.        ,  0.01346821,
       -0.01346254,  0.01195898,  0.00156612])

In [11]:
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

Now print the 10th row (index 9) of the training feature matrix. Again, you get an 18-dimensional vector with components between 0 and 1.

***QUIZ QUESTION ***

What is the Euclidean distance between the query house and the 10th house of the training set? 

Note: Do not use the `np.linalg.norm` function; use `np.sqrt`, `np.sum`, and the power operator (`**`) instead. The latter approach is more easily adapted to computing multiple distances at once.

In [12]:
distance = np.sqrt(np.sum((features_test[0]-features_train[9])**2))

In [13]:
distance

0.059723593716661257

In [14]:
np.linalg.norm(features_test[0]-features_train[9])

0.059723593716661257

In [15]:
dist(features_test[0],features_train[9])

0.059723593716661257

# Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and *all* houses in the training set.  

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses.  Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

In [16]:
min=1.0
minindx=0
for i in range(10):
    d = dist(features_test[0], features_train[i])
    if d < min:
        min = d
        minindx=i
    print (d, i)
print ("min ", min, minindx)

(0.0602747091729555, 0)
(0.085468811488270832, 1)
(0.061499464371202843, 2)
(0.053402739788200579, 3)
(0.058444840639381393, 4)
(0.059879215101840008, 5)
(0.054631404972615261, 6)
(0.055431083241597921, 7)
(0.052383627840972731, 8)
(0.059723593716661257, 9)
('min ', 0.052383627840972731, 8)


In [87]:
train[8]['price']

937000

In [86]:
train[0]

{'bathrooms': 1.0,
 'bedrooms': 3.0,
 'condition': 3,
 'constant': 1,
 'date': datetime.datetime(2014, 10, 13, 0, 0, tzinfo=GMT +0.0),
 'floors': 1.0,
 'grade': 7,
 'id': '7129300520',
 'lat': 47.51123398,
 'long': -122.25677536,
 'price': 221900,
 'sqft_above': 1180,
 'sqft_basement': 0,
 'sqft_living': 1180.0,
 'sqft_living15': 1340.0,
 'sqft_lot': 5650,
 'sqft_lot15': 5650.0,
 'view': 0,
 'waterfront': 0,
 'yr_built': 1955,
 'yr_renovated': 0,
 'zipcode': '98178'}

*** QUIZ QUESTION ***

Among the first 10 training houses, which house is the closest to the query house?

In [17]:
9th house

SyntaxError: invalid syntax (<ipython-input-17-8cd5db3be535>, line 1)

It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the Numpy functions can be **vectorized**, applying the same operation over multiple values or vectors.  We now walk through this process.

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [18]:
for i in xrange(3):
    print features_train[i]-features_test[0]
    # should print 3 vectors of length 18

[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
   2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
  -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
  -5.14364795e-03   6.69281453e-04]
[  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
   7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
  -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
  -2.89330197e-03   1.47606982e-03]
[  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
   1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
  -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
   3.72914476e-03   1.64764925e-03]


The subtraction operator (`-`) in Numpy is vectorized as follows:

In [19]:
print features_train[0:3] - features_test[0]

[[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
    2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
   -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
   -5.14364795e-03   6.69281453e-04]
 [  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
    7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
   -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
   -2.89330197e-03   1.47606982e-03]
 [  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
    1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
   -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
    3.72914476e-03   1.64764925e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [20]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print results[0] - (features_train[0]-features_test[0])
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print results[1] - (features_train[1]-features_test[0])
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print results[2] - (features_train[2]-features_test[0])
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [21]:
results 

array([[  0.00000000e+00,  -3.87821276e-03,  -1.20498190e-02,
         -1.05552733e-02,   2.08673616e-04,  -8.52950206e-03,
          0.00000000e+00,  -5.10236549e-02,   0.00000000e+00,
         -3.47633726e-03,  -5.50336860e-03,  -2.48168183e-02,
         -1.63756198e-04,   0.00000000e+00,  -1.70072004e-05,
          1.30577772e-05,  -5.14364795e-03,   6.69281453e-04],
       [  0.00000000e+00,  -3.87821276e-03,  -4.51868214e-03,
         -2.26610387e-03,   7.19763456e-04,   0.00000000e+00,
          0.00000000e+00,  -5.10236549e-02,   0.00000000e+00,
         -3.47633726e-03,   1.30705004e-03,  -1.45830788e-02,
         -1.91048898e-04,   6.65082271e-02,   4.23240653e-05,
          6.22415897e-06,  -2.89330197e-03,   1.47606982e-03],
       [  0.00000000e+00,  -7.75642553e-03,  -1.20498190e-02,
         -1.30002801e-02,   1.60518166e-03,  -8.52950206e-03,
          0.00000000e+00,  -5.10236549e-02,   0.00000000e+00,
         -5.21450589e-03,  -8.32384500e-03,  -2.48168183e-02,
      

Aside: it is a good idea to write tests like this cell whenever you are vectorizing a complicated operation.

# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the `i`-th training house.

In [22]:
diff = features_train[:] - features_test[0]

In [154]:
output_valid

array([385000, 285000, 687500, ..., 329000, 362500, 429000])

To test the code above, run the following cell, which should output a value -0.0934339605842:

In [23]:
print diff[-1].sum() # sum of the feature differences between the query and last training house
# should print -0.0934339605842

-0.0934339605842


The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices.  That is, compute the sum of square feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` [documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html). In particular, `axis=1` computes the sum across each row.

Below, we compute this sum of square feature differences for all training houses and verify that the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

In [24]:
print np.sum(diff**2, axis=1)[15] # take sum of squares across each row, and print the 16th sum
print np.sum(diff[15]**2) # print the sum of squares for the 16th row -- should be same as above

0.00330705902879
0.00330705902879


With this result in mind, write a single-line expression to compute the Euclidean distances between the query house and all houses in the training set. Assign the result to a variable `distances`.

**Hint**: Do not forget to take the square root of the sum of squares.

In [25]:
distances = np.sqrt(np.sum(diff**2, axis=1))[:]

In [26]:
distances[100]

0.023708232449603735

To test the code above, run the following cell, which should output a value 0.0237082324496:

In [27]:
print distances[100] # Euclidean distance between the query house and the 101th training house
# should print 0.0237082324496

0.0237082324496


Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [28]:
def distNN(feature, query):
    diff = feature[:] - query
    return np.sqrt(np.sum(diff**2, axis=1))[:]

*** QUIZ QUESTIONS ***

1.  Take the query house to be third house of the test set (`features_test[2]`).  What is the index of the house in the training set that is closest to this query house?
2.  What is the predicted value of the query house based on 1-nearest neighbor regression?

In [29]:
dist2 = distNN(features_train, features_test[2])

In [32]:
min =1.0
minidx=0
for i in range(len(dist2)):
    if min > dist2[i]:
        min = dist2[i]
        minidx = i
print (min, minidx)

(0.0028604952675079271, 382)


In [33]:
features_train[382]

array([ 0.01345102,  0.01163464,  0.00903736,  0.01013783,  0.00264759,
        0.0085295 ,  0.        ,  0.        ,  0.0116321 ,  0.01216718,
        0.006948  ,  0.0176532 ,  0.01344165,  0.        ,  0.01341151,
       -0.01344709,  0.00925857,  0.00340725])

In [34]:
dist2

array([ 0.01954476,  0.06861035,  0.02165079, ...,  0.02433478,
        0.02622734,  0.02637942])

In [35]:
len(dist2)

5527

In [36]:
dist2[1]

0.068610347115025894

In [37]:
train[382]['price']

249000

# Perform k-nearest neighbor regression

For k-nearest neighbors, we need to find a *set* of k houses in the training set closest to a given query house. We then make predictions based on these k nearest neighbors.

## Fetch k-nearest neighbors

Using the functions above, implement a function that takes in
 * the value of k;
 * the feature matrix for the training houses; and
 * the feature vector of the query house
 
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

**Hint**: Look at the [documentation for `np.argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

In [38]:
tempkNN = distNN(features_train[0:3], features_test[1])

In [39]:
tempkNN

array([ 0.00815217,  0.06844661,  0.01449589])

In [40]:
srt = np.argsort(tempkNN)

In [41]:
srt2 = np.sort(tempkNN)

In [42]:
srt2

array([ 0.00815217,  0.01449589,  0.06844661])

In [43]:
srt

array([0, 2, 1])

In [44]:
tempkNN[srt[2]]

0.068446612674472898

In [45]:
def dist1NN(feature, query):
    diff = feature - query
    return np.sqrt(np.sum(diff**2))

In [50]:
def shiftX(sortedkNN, s_distkNN, currdist, i, j,  k):
    #shift the rest
    for index in range(k-2,j-1, -1):
        sortedkNN[index+1]=  sortedkNN[index]
        s_distkNN[index+1] = s_distkNN[index]
    #set the sortedkNN and distkNN array with the  new min val index and distance 
    sortedkNN[j]=i
    s_distkNN[j]=currdist
    #print ("shifted ", sortedkNN, s_distkNN)
    return
    

In [71]:
distkNN = distNN(features_train[:], features_test[2])

In [73]:
 sortedkNN = np.argsort(distkNN)
print sortedkNN[:10], distkNN

[ 382 1149 4087 3142 2751 4556 3372 4584 1666 2283] [ 0.01954476  0.06861035  0.02165079 ...,  0.02433478  0.02622734
  0.02637942]


In [66]:
curr_dist = dist1NN(features_train[6], features_test[2])
print curr_dist

0.00543398935931


In [54]:
curr_dist

0.0032258403880432848

In [97]:
def kNN(features_train, query, k):
    distkNN = distNN(features_train[0:k], query)
    #print distkNN
    s_distkNN = np.sort(distkNN)
    sortedkNN = np.argsort(distkNN)
    for i in range(k, len(features_train)):
        curr_dist = dist1NN(features_train[i], query)
        if curr_dist < s_distkNN[k-1]:
            #print ("sorted ", s_distkNN[k-1] )
            for j in range(k):
                if curr_dist < s_distkNN[j]:
                    #print ("before ", sortedkNN, s_distkNN, curr_dist, i, j)
                    shiftX(sortedkNN, s_distkNN, curr_dist, i, j,  k)
                    #print ("after ", sortedkNN, s_distkNN, curr_dist, i, j)
                    break
    #print "sorted kNN", sortedkNN
    #print "distances ", s_distkNN
    return sortedkNN
        

*** QUIZ QUESTION ***

Take the query house to be third house of the test set (`features_test[2]`).  What are the indices of the 4 training houses closest to the query house?

In [79]:
fourNN = kNN(features_train[:], features_test[2], 4)

sorted kNN [ 382 1149 4087 3142]
distances  [ 0.0028605   0.00322584  0.00350216  0.00359315]


In [75]:
fourNN

array([ 382, 1149, 4087, 3142])

In [80]:
len(fourNN)

4

In [91]:
train[382]['price']

249000

In [92]:
train[1149]['price']

477000

In [93]:
train[4087]['price']

499950

In [94]:
train[3142]['price']

430000

In [160]:
def getKAvgPrice(karray, train):
    avg=0.0
    for i in karray:
        avg += train[i]['price']
    
    #print avg/len(karray)
    return avg/len(karray)
        

In [89]:
price = getKAvgPrice(fourNN, train)

413987.5


## Make a single prediction by averaging k nearest neighbor outputs

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. **For simplicity, take the average of the prices of the k nearest neighbors in the training set**. The function should have the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature vector of the query house, whose price we are predicting.
 
The function should return a predicted value of the query house.

**Hint**: You can extract multiple items from a Numpy array using a list of indices. For instance, `output_train[[6, 10]]` returns the prices of the 7th and 11th training houses.

*** QUIZ QUESTION ***

Again taking the query house to be third house of the test set (`features_test[2]`), predict the value of the query house using k-nearest neighbors with `k=4` and the simple averaging method described and implemented above.

Compare this predicted value using 4-nearest neighbors to the predicted value using 1-nearest neighbor computed earlier.

## Make multiple predictions

Write a function to predict the value of *each and every* house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature matrix for the query set.
 
The function should return a set of predicted values, one for each house in the query set.

**Hint**: To get the number of houses in the query set, use the `.shape` field of the query features matrix. See [the documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ndarray.shape.html).

In [104]:
def kNNPrediction(features_train, query_set,  k):
    indx =0
    min_price = 10000000000
    min_indx = 0
    for query in query_set:
        KNN = kNN(features_train, query, k)
        price = getKAvgPrice(KNN, train)
        output[indx] = price
        print indx, price
        if min_price > price:
            min_price = price
            min_indx = indx
        indx +=1
    print "min ", min_indx, min_price
    

*** QUIZ QUESTION ***

Make predictions for the first 10 houses in the test set using k-nearest neighbors with `k=10`. 

1. What is the index of the house in this query set that has the lowest predicted value? 
2. What is the predicted value of this house?

In [105]:
kNNPrediction(features_train, features_test[0:10], 10)

881300.0
0 881300.0
431860.0
1 431860.0
460595.0
2 460595.0
430200.0
3 430200.0
766750.0
4 766750.0
667420.0
5 667420.0
350032.0
6 350032.0
512800.7
7 512800.7
484000.0
8 484000.0
457235.0
9 457235.0
min  6 350032.0


In [127]:
def kNNPredictionTest(features_train, query_set, output,  k):
    indx =0
    min_price = 10000000000
    min_indx = 0
    for query in query_set:
        KNN = kNN(features_train, query, k)
        price = getKAvgPrice(KNN, train)
        output[indx] = price
        indx +=1
    return output

In [128]:
kNNPredictionTest(features_train, features_test[0:10], output, 5)

878000.0
418520.0
381190.0
530200.0
869700.0
697590.0
332764.0
530000.0
493000.0
449780.0


array([ 878000.,  418520.,  381190.,  530200.,  869700.,  697590.,
        332764.,  530000.,  493000.,  449780.])

In [129]:
output= np.zeros(10)

In [125]:
output

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [131]:
predict =kNNPredictionTest(features_train, features_test[0:10], output, 10)

881300.0
431860.0
460595.0
430200.0
766750.0
667420.0
350032.0
512800.7
484000.0
457235.0


In [133]:
error = predict - test[0:10]['price']

In [138]:
def RSS(predict, actual):
    error = predict - actual
    return np.sum(error*error)

In [139]:
RSS(predict, test[0:10]['price'])

227424136694.48999

In [115]:
output

array(10)

In [147]:
def kNNPredictionSet(features_train, query_set,  k):
    indx =0
    output = np.zeros(len(query_set))
    for query in query_set:
        KNN = kNN(features_train, query, k)
        price = getKAvgPrice(KNN, train)
        output[indx] = price
        indx +=1
    return output

In [148]:
predict =kNNPredictionSet(features_train, features_test[0:10], 10)

881300.0
431860.0
460595.0
430200.0
766750.0
667420.0
350032.0
512800.7
484000.0
457235.0


In [149]:
predict

array([ 881300. ,  431860. ,  460595. ,  430200. ,  766750. ,  667420. ,
        350032. ,  512800.7,  484000. ,  457235. ])

## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

* For `k` in [1, 2, ..., 15]:
    * Makes predictions for each house in the VALIDATION set using the k-nearest neighbors from the TRAINING set.
    * Computes the RSS for these predictions on the VALIDATION set
    * Stores the RSS computed above in `rss_all`
* Report which `k` produced the lowest RSS on VALIDATION set.

(Depending on your computing environment, this computation may take 10-15 minutes.)

In [164]:
def kNNPrediction(features_train, query_set, output):
    min_rss = 10000000000000000
    min_k = 0
    size = len(query_set)
    for k in range(1,16):
        predictions = kNNPredictionSet(features_train, query_set, k)
        rss = RSS(predictions, output)
        if min_rss > rss:
            min_rss = rss
            min_k = k
        print rss, k 
    print ("min  ", rss, k)

In [165]:
kNNPrediction(features_train, features_test, output_test)

1.89163641317e+14 1
1.62999233174e+14 2
1.49035721335e+14 3
1.37922415887e+14 4
1.32306923253e+14 5
1.32717697867e+14 6
1.31774119937e+14 7
1.33091689367e+14 8
1.33492116715e+14 9
1.32558337003e+14 10
1.32648119251e+14 11


KeyboardInterrupt: 

In [166]:
kNNPrediction(features_train, features_valid, output_valid)

1.05451197752e+14 1
8.3445073504e+13 2
7.26920960192e+13 3
7.19348033496e+13 4
6.98465174197e+13 5
6.8903104922e+13 6
6.83383144186e+13 7
6.73616787355e+13 8
6.8372727959e+13 9
6.93335579816e+13 10
6.95238552156e+13 11
6.90519486845e+13 12
7.00112545083e+13 13
7.09115306803e+13 14
7.11087974867e+13 15
('min  ', 71108797486656.281, 15)


In [ ]:
rss_all = np.array(15)

In [ ]:
rss_all[0]=1.05451197752e+14

In [ ]:
rss_all[1]=8.3445073504e+13

In [ ]:
rss_all[2]=7.26920960192e+13

To visualize the performance as a function of `k`, plot the RSS on the VALIDATION set for each considered `k` value:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

kvals = range(1, 16)
plt.plot(kvals, rss_all,'bo-')

***QUIZ QUESTION ***

What is the RSS on the TEST data using the value of k found above?  To be clear, sum over all houses in the TEST set.

In [170]:
predictions_test = kNNPredictionSet(features_train, features_test, 8)
rss_test = RSS(predictions_test, output_test)

In [171]:
rss_test

133091689366516.81

In [168]:
rss_test =RSS(predictions_test, output_test)

In [169]:
rss_test

4911549456047381.0